In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:

# Connect to database
# Using local path assuming notebook is in the same directory as the sqlite file
db_path = "football_database.sqlite"
connection = sqlite3.connect(db_path)

In [ ]:
# Load dataframes
df_country = pd.read_sql_query("SELECT * FROM Country", connection)
df_league = pd.read_sql_query("SELECT * FROM League", connection)
df_match = pd.read_sql_query("SELECT * FROM Match", connection)
df_player = pd.read_sql_query("SELECT * FROM Player", connection)
df_player_attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", connection)
df_team = pd.read_sql_query("SELECT * FROM Team", connection)
df_team_attributes = pd.read_sql_query("SELECT * FROM Team_Attributes", connection)

In [23]:
# Print first few lines of key dataframes
print("--- Country ---")
display(df_country.head())

print("\n--- League ---")
display(df_league.head())

print("\n--- Match ---")
display(df_match.head())

print("\n--- Player ---")
display(df_player.head())



--- Country ---


,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy



--- League ---


,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A



--- Match ---


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67



--- Player ---


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [ ]:
draft_db_path = "../draft_ministers.db"
conn_draft = sqlite3.connect(draft_db_path)

# Extract teams from soccer_teams table
df_draft_teams = pd.read_sql_query("SELECT * FROM soccer_teams", conn_draft)
print("--- Draft Ministers Teams ---")
display(df_draft_teams.head())

conn_draft.close()

In [22]:
# Find df_draft_teams in df_team
# First, let's check the column names to understand how to match
print("--- df_team columns ---")
print(df_team.columns.tolist())
print("\n--- df_draft_teams columns ---")
print(df_draft_teams.columns.tolist())

# Match teams by name (case-insensitive)
# Normalize team names for better matching
df_team['team_name_normalized'] = df_team['team_long_name'].str.lower().str.strip()
df_draft_teams['name_normalized'] = df_draft_teams['name'].str.lower().str.strip()

# Find matches
matched_teams = df_team[df_team['team_name_normalized'].isin(df_draft_teams['name_normalized'])]
print(f"\n--- Found {len(matched_teams)} matching teams ---")
display(matched_teams[['id', 'team_api_id', 'team_long_name', 'team_short_name']])

# Show which draft teams were found
draft_teams_found = df_draft_teams[df_draft_teams['name_normalized'].isin(df_team['team_name_normalized'])]
print(f"\n--- {len(draft_teams_found)} draft teams found in df_team ---")
display(draft_teams_found[['id', 'name', 'code', 'country', 'league']])

# Show draft teams NOT found
draft_teams_not_found = df_draft_teams[~df_draft_teams['name_normalized'].isin(df_team['team_name_normalized'])]
print(f"\n--- {len(draft_teams_not_found)} draft teams NOT found in df_team ---")
display(draft_teams_not_found[['id', 'name', 'code', 'country', 'league']])


--- df_team columns ---
['id', 'team_api_id', 'team_fifa_api_id', 'team_long_name', 'team_short_name', 2379689825376]

--- df_draft_teams columns ---
['id', 'name', 'code', 'country', 'founded', 'national', 'logo', 'venue_id', 'venue_name', 'venue_address', 'venue_city', 'venue_capacity', 'venue_surface', 'venue_image', 'league']

--- Found 11 matching teams ---


,id,team_api_id,team_long_name,team_short_name
25,3457,10260,Manchester United,MUN
27,3459,9825,Arsenal,ARS
30,3462,8650,Liverpool,LIV
33,3465,10252,Aston Villa,AVL
34,3466,8456,Manchester City,MCI
35,3467,8668,Everton,EVE
42,3474,9879,Fulham,FUL
43,3475,8455,Chelsea,CHE
47,4234,8191,Burnley,BUR
54,7261,9826,Crystal Palace,CRY



--- 11 draft teams found in df_team ---


,id,name,code,country,league
0,33,Manchester United,MUN,England,Premier League
2,35,Bournemouth,BOU,England,Premier League
3,36,Fulham,FUL,England,Premier League
5,40,Liverpool,LIV,England,Premier League
6,42,Arsenal,ARS,England,Premier League
7,44,Burnley,BUR,England,Premier League
8,45,Everton,EVE,England,Premier League
11,49,Chelsea,CHE,England,Premier League
12,50,Manchester City,MAC,England,Premier League
14,52,Crystal Palace,CRY,England,Premier League



--- 9 draft teams NOT found in df_team ---


,id,name,code,country,league
1,34,Newcastle,NEW,England,Premier League
4,39,Wolves,WOL,England,Premier League
9,47,Tottenham,TOT,England,Premier League
10,48,West Ham,WES,England,Premier League
13,51,Brighton,BRI,England,Premier League
15,55,Brentford,BRE,England,Premier League
16,62,Sheffield Utd,SHE,England,Premier League
17,65,Nottingham Forest,NOT,England,Premier League
19,1359,Luton,LUT,England,Premier League
